# Burton's Geomagnetic Storm Model

This notebook will be an implementation of the algorithm outlined by [Rande Burton](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/JA080i031p04204) on the prediction of Dst from solar wind parameters as provided by the OMNI dataset.

In [1]:
import numpy as np
import pandas as pd

## Load Data

Firstly, I'll need the OMNI data, since Burton's model depends entirely on the following measurements of solar wind data:

- V: velocity (magnitude)
- n: proton density
- $B_z$: north-south component of the interplanetary magnetic field (IMF)

Since I'm not dependent on continuous time series to the extent that I am with training predictive networks, I'll use the data with gaps and just ignore the gaps.

In [2]:
omni = pd.read_hdf('omni_hourly_1998-2018.h5')

# let's only keep the following columns
omni = omni[['SW Plasma Speed, km/s', 'BZ, nT (GSM)','SW Proton Density, N/cm^3', 'Dst-index, nT']]

# list gaps by year for each variable and total at end
nanvals = np.max(omni,axis=0)

# replace nanval for Dst, no missing data there
nanvals['Dst-index, nT'] = 999.9

# compute gap free indices
idx = np.ones(len(omni), dtype=bool)
for ii, meas in enumerate(list(omni)):
    idx = idx & (omni[meas] != nanvals[meas])

# ignore gaps here
omni = omni[idx]

V = omni['SW Plasma Speed, km/s']
Bz = omni['BZ, nT (GSM)']
n = omni['SW Proton Density, N/cm^3']
Dst = omni['Dst-index, nT']

## Theory

From the primary measurements, Burton defines the electric field $E = V\,B_z$ and the dynamic pressure $P = n\, V^2$.

In [42]:
E = v * bz
P = n * v**2

Dst is defined as the difference between the disturbance and quiet time measurements of the horizontal field component at low latitudes:

$$ Dst = \overline{H^d - H^q} $$

Burton furthermore models Dst in terms of its components from the magnetopause and ring currents, which have both disturbance and quiet time components:

$$ Dst = \overline{H^d_{mp}} + \overline{H^d_{rc}} -\overline{H^q_{mp}} - \overline{H^q_{rc}} $$

After introducing this separation in sources, Burton then renames the ring current contribution to Dst as $Dst_0 = \overline{H^d_{rc}}$ and posits that the quiet time contributions can be modeled as a constant $ c = \overline{H^q_{mp}} + \overline{H^q_{rc}}$. Finally, the magnetopause current contribution to Dst is proportional to square root of the dynamic pressure of the solar wind (see his citations), $\overline{H^d_{mp}} = b\,\sqrt{P}$.

Thus, the relation for $Dst_0$ can be rewritten as 

$$ Dst_0 = Dst - b\,\sqrt{P} + c$$

Since $Dst_0$ is then tied to the ring current contribution, if we can model its physical relationship to the solar wind, we can reconstruct Dst entirely from solar wind measurements (assuming that we've constrained the necessary constants).

Burton argues that the ring current contribution to Dst then depends on an injection term $F(E)$ dependent on the generated electric field that drives the main phase of the storm and subsequent decay, which in turn is proportional to the current ring current strength (setting up an exponential decay):

$$ \frac{d Dst_0}{dt} = F(E) - a\,Dst_0 $$

Thus, rewriting everything in terms of $Dst_0$, we can write the total time rate of change of Dst as 

$$ \frac{d Dst}{dt} = -a \left(Dst - b\,\sqrt{P} + c \right) + F(E) + \frac{d}{dt}\left(b\,\sqrt{P}\right) $$

An appropriate time delay must be introduced to account for the distance between where solar wind parameters are measured and Earth's surface, but the OMNI data already account for this separation.

## Parameter Estimation

Thus, the next step in forecasting Dst is determining the corresponding constants $a$, $b$, and $c$ as well as the ring current forcing $F(E)$ which will depend on the generated electric field filtered by some unknown frequency response $H(\omega)$.

Burton proceeds by considering the asymptotics of the above equation, arriving at the following values that they use for the constants:

- $a = $
- $b = 0.20 $
- $c = 20 $

I will try to recreate some of these estimates to see if they vary at all given that I have access to data gathered 40 years after the publication of their paper.


### Quiet time estimation

Here, I'll take the approach that assumes no ring current injection (i.e. $F(E) = 0$) and constant dynamic pressure (i.e. $\frac{dP}{dt} = 0 $), so that the relevant equation is 

$$ \frac{d Dst}{dt} = -a \, \left(Dst - b\,\sqrt{P} + c \right) $$

This expression will asymptotically approach zero, and so eventually Dst will approach the value satisfying 

$$ Dst - b\,\sqrt{P} + c = 0$$

allowing us to solve for $b$ and $c$. This expression is true during quiet times, which we can isolate from the data above.

In [ ]:
# isolate data from quiet times
idx = 